In [1]:
import os
from ftplib import FTP, FTP_TLS
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import h5py
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

from datetime import datetime, time, timedelta
from joblib import Parallel, delayed
from mpl_toolkits.basemap import Basemap

In [2]:
output_data_dir = '/export/data1/cchristo/gpm_data' # no trailing backslash!

In [3]:
ftp = FTP('arthurhou.pps.eosdis.nasa.gov')
# ftp = FTP_TLS('jsimpsonftps.pps.eosdis.nasa.gov')
# ftp = FTP_TLS('arthurhou.pps.eosdis.nasa.gov')
ftp.login('cchristo@caltech.edu','cchristo@caltech.edu')

'230 User cchristo@caltech.edu logged in'

In [ ]:
# ftp = FTP('arthurhou.pps.eosdis.nasa.gov')
# ftp.login('cchristo@caltech.edu','cchristo@caltech.edu')

In [4]:
ftp.nlst()

['trmmdata',
 'trmmopen',
 'robots.txt',
 'ftpdata',
 'gpmdata',
 'gpmuser',
 'gpmallversions',
 'sm',
 'pub',
 'itedata']

In [5]:
ftp.cwd('/gpmdata/')
ftp.nlst()

['coincidence',
 'geolocation',
 '2014',
 'browse',
 '1999',
 'manifest',
 '2007',
 '2006',
 '2008',
 '2016',
 '2017',
 '1997',
 '1998',
 'zonals',
 '2011',
 '2012',
 '2013',
 '2000',
 '2015',
 'trending',
 'README.TXT',
 'documents',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2009',
 '2010',
 '2018',
 '1991',
 '1995',
 '1987',
 '1990',
 '1992',
 '1988',
 '1996',
 '1993',
 '1989',
 '1994',
 '2019',
 '2020']

In [55]:
ftp.cwd('/gpmdata/2001/01/01/imerg')
ftp.nlst()
# ftp.cwd('/gpmdata/2000/05/01')

['3B-HHR.MS.MRG.3IMERG.20010101-S213000-E215959.1290.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S203000-E205959.1230.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S183000-E185959.1110.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S040000-E042959.0240.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S033000-E035959.0210.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S110000-E112959.0660.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S023000-E025959.0150.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S233000-E235959.1410.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S200000-E202959.1200.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S220000-E222959.1320.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S090000-E092959.0540.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S190000-E192959.1140.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S020000-E022959.0120.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S050000-E052959.0300.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20010101-S153000-E155959.0930.V06B.HDF5',
 '3B-HHR.MS.MRG.3IMERG.20

In [24]:
# sorted(ftp.nlst())
# ['3B-HHR.MS.MRG.3IMERG.20010101-S000000-E002959.0000.V06B.HDF5',

In [ ]:
# download all files in a daterange 

In [ ]:
# for model_name in model_name_to_http_filename_map:
#     print(model_name)
#     all_urls_to_download = model_name_to_http_filename_map[model_name]
#     for url_to_download in all_urls_to_download:
#         local_filepath = output_data_dir + model_name + '/' + url_to_download.split('/')[-1]
#         print(local_filepath)
#         if not os.path.exists(local_filepath):
#             print("Does not exist", local_filepath)
#             result_code = os.system('wget -nc -c --retry-connrefused --waitretry=10 --quiet -P ' + output_data_dir + model_name + '/' + ' ' + url_to_download)
#     print('Done!')

In [31]:
# dates = pd.date_range(start = '2000-06-01', end = '2011-06-01')
# dates = pd.date_range(start = '2000-06-01', end = '2011-06-01')
dates = pd.date_range(start = '2011-06-01', end = '2020-06-01')

In [42]:
# dates[0].strftime('%Y/%M/%d')
retry_connection_counter = 0
for date_i in dates:
#     ftp.cwd('/gpmdata/2009/01/01/imerg')
    try:
        rel_dir_to_files = date_i.strftime('/gpmdata/%Y/%m/%d/imerg')
        logging.info(f'Downloading: {rel_dir_to_files}')
        try:
            ftp.cwd(rel_dir_to_files)
        except: 
            logging.info(f'Error with {rel_dir_to_files}!')


        sorted_fnames = sorted(ftp.nlst())

        out_file_rel_path = output_data_dir + rel_dir_to_files

        # if dir for month/day doesn't exist, create it 
        if not os.path.exists(out_file_rel_path):
            os.makedirs(out_file_rel_path)

            for fname in sorted_fnames:
                try:
                    out_file_path = out_file_rel_path + '/' + fname
                    if not os.path.exists(out_file_path):
                        with open(out_file_path, 'wb') as out_file: # Open a local file to store the downloaded file
                            print(fname)
                            ftp.retrbinary('RETR ' + fname, out_file.write, 8*1024) # Enter the filename to download
                        # open and convert to NETCDF, then save again
                        hf = h5py.File(out_file_path, 'r')
                        raw_data = hf['Grid']
                        raw_data_np = raw_data.get('precipitationCal')[()]
                        lat_np = raw_data.get('lat')[()]
                        lon_np = raw_data.get('lon')[()]
                        time_np = raw_data.get('time')[()]
                        time_np = np.array(datetime.utcfromtimestamp(time_np.item())).reshape([1])

                        # make numpy arrays into dataarray 
                        da = xr.DataArray(raw_data_np.T, dims = ('lat', 'lon', 'time'), 
                              coords = {'time': time_np,
                                        'lon': lon_np,
                                        'lat': lat_np})
                        da.name = 'precipitationCal'

                        # save as netcdf
                        new_out_file_path = out_file_rel_path + '/'+ fname[:-5] + '.nc'
                        da.to_netcdf(new_out_file_path)

                        # delete hdf5 file if it exists 
                        if os.path.exists(out_file_path):
                            os.remove(out_file_path)
                except Exception as e: 
                    logging.info(f'Failed on : {out_file_path}')
                    logging.info(f'Reason : {str(e)}')
                
            
    except Exception as f:
        logging.info(f'Failed on date : {str(date_i)}')
        logging.info(f'Reason : {str(f)}')
        
        retry_connection_counter += 1
        if retry_connection_counter < 10: 
            ftp = FTP('arthurhou.pps.eosdis.nasa.gov')
            ftp.login('cchristo@caltech.edu','cchristo@caltech.edu')
            

    

INFO:root:Downloading: /gpmdata/2000/06/01/imerg
INFO:root:Downloading: /gpmdata/2000/06/02/imerg
  0%|          | 0/48 [00:00<?, ?it/s]

3B-HHR.MS.MRG.3IMERG.20000602-S000000-E002959.0000.V06B.HDF5


  2%|▏         | 1/48 [00:14<11:29, 14.68s/it]

3B-HHR.MS.MRG.3IMERG.20000602-S003000-E005959.0030.V06B.HDF5


  4%|▍         | 2/48 [00:18<08:47, 11.46s/it]

...

 96%|█████████▌| 46/48 [02:45<00:06,  3.26s/it]

3B-HHR.MS.MRG.3IMERG.20000602-S230000-E232959.1380.V06B.HDF5


 98%|█████████▊| 47/48 [02:48<00:03,  3.27s/it]

3B-HHR.MS.MRG.3IMERG.20000602-S233000-E235959.1410.V06B.HDF5


100%|██████████| 48/48 [02:52<00:00,  3.58s/it]


In [37]:
# ftp.pwd()
# out_file_path.split('/')[:-1]


In [45]:
out_file_path

'/export/data1/cchristo/gpm_data/gpmdata/2000/06/02/imerg/3B-HHR.MS.MRG.3IMERG.20000602-S233000-E235959.1410.V06B.HDF5'

In [ ]:
# ftp = FTP('arthurhou.pps.eosdis.nasa.gov')
# ftp.login('cchristo@caltech.edu','cchristo@caltech.edu')

In [44]:
# !wget -r "ftp://cchristo@caltech.edu:cchristo@caltech.edu@arthurhou.pps.eosdis.nasa.gov/sm/730/gpmdata/2000/06/01/imerg"


out_file_rel_path



'/export/data1/cchristo/gpm_data/gpmdata/2000/06/02/imerg'

In [56]:
# listing = []
# ftp.retrlines("LIST", listing.append)
# words = listing[0].split(None, 8)
# filename = words[-1].lstrip()
# filename = '/sm/730/gpmdata/2000/06/01/imerg/3B-HHR.MS.MRG.3IMERG.20000601-S000000-E002959.0000.V06B.HDF5'

In [60]:
# filename
# words
# listing
# words

In [39]:
# listing[0]

In [ ]:
# wget --user=cchristo@caltech.edu --password=cchristo@caltech.edu ftp://arthurhou.pps.eosdis.nasa.gov/sm/730/gpmdata/2000/06/01/imerg/3B-HHR.MS.MRG.3IMERG.20000601-S000000-E002959.0000.V06B.HDF5

In [62]:
# from nasadap import agg

# ###############################
# ### Parameters

# cache_dir = 'nasa/cache/nz'
# save_dir = 'nasa/precip'

# username = 'cos14' # Need to change!
# password = 'Clouds14' # Need to change!

# mission = 'gpm'
# freq = 'M'
# product = '3IMERGHH'
# version = 6
# datasets = ['precipitationCal']

# min_lat=-49
# max_lat=-33
# min_lon=165
# max_lon=180
# dl_sim_count = 50
# tz_hour_gmt = 12

# agg.time_combine(mission, product, version, datasets, save_dir, username, password,
#                   cache_dir, tz_hour_gmt, freq, min_lat, max_lat, min_lon,
#                   max_lon, dl_sim_count)